In [1]:
import os
import sys

sys.path.append("../")
adaptive_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if adaptive_root not in sys.path:
    sys.path.append(adaptive_root)

In [2]:
import logging
from copy import deepcopy

import numpy as np
from cmdstanpy import CmdStanModel
from IPython.display import display
from simulation import aggregators, estimators, fitters, results, simulators, utils

logger = logging.getLogger("cmdstanpy")
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In this notebook, we examine the RMSE of our estimators when varying $\sigma_0$ and $\sigma_1$.

In [3]:
model = CmdStanModel(stan_file="../basic_mlm.stan")
model_hp = CmdStanModel(stan_file="../basic_mlm_hp.stan")

exposed_only = estimators.ExposedOnlyEstimator()
difference = estimators.DifferenceEstimator()

posterior_mean_fitter = fitters.StanFitterBasic(model)

posterior_mean = estimators.PosteriorMeanEstimator(posterior_mean_fitter)

posterior_mean_est_hp_fitter = fitters.StanFitterWithEstimatedHyperparams(
    model_hp, fitters.StanFitterBasic(model)
)

posterior_mean_est_hp = estimators.PosteriorMeanEstimator(posterior_mean_est_hp_fitter)
posterior_mean_est_hp.name = "Posterior Mean (Estimated HP)"

static_simulator = utils.CHICK_SIMULATOR
J = len(static_simulator.n)
p_star_oracle = simulators.AdaptiveExperimentSimulator(
    **static_simulator.params,
    J=J,
    p_callback=utils.next_p_star,
    n_callback=lambda expt: static_simulator.n[0],
    identifier={"name": "p_star_oracle"},
)
p_star_estimated = simulators.AdaptiveExperimentSimulator(
    **static_simulator.params,
    J=J,
    p_callback=lambda expt, n: utils.next_p_star(expt, n, oracle=True),
    n_callback=lambda expt: static_simulator.n[0],
    identifier={"name": "p_star_estimated"},
)


def p_callback_hp(expt, n):
    if expt is None:
        return 0.5
    sigma0 = expt.params["sigma0"]
    sigma1 = expt.params["sigma1"]

    result = posterior_mean_fitter.fit(expt, show_progress=False)
    sigma_b_hat = np.mean(result.sigma_b)
    numer = 1 + sigma0**2 / (n * sigma_b_hat**2)
    denom = 1 + sigma0 / sigma1
    p_star = numer / denom

    print(f"HP: {min(0.95, p_star)}")

    return min(0.95, p_star)


p_star_estimated_hp = simulators.AdaptiveExperimentSimulator(
    **static_simulator.params,
    J=J,
    p_callback=p_callback_hp,
    n_callback=lambda expt: static_simulator.n[0],
    identifier={"name": "p_star_estimated_hp"},
)

In [4]:
original_sims = [
    static_simulator,
    p_star_oracle,
    p_star_estimated,
]

ests = [
    posterior_mean,
    posterior_mean_est_hp,
    exposed_only,
    difference,
]

The cell below fixes $\sigma_b$ to what its value for the original chicken data and $\sigma_0 = \sigma_1$ are modified to produce a value of $R = 1/4$, where $R := \sigma_0^2 / (n \sigma_b^2)$.

In [ ]:
import copy

modified_sims = copy.deepcopy(original_sims)
for sim in modified_sims:
    sim.params["sigma0"] = np.sqrt(64 * utils.CHICK_SIGMA_B**2) / 2
    sim.params["sigma1"] = sim.params["sigma0"]

In [ ]:
aggregator = aggregators.SimulationAggregator(simulators=original_sims)

In [7]:
result = aggregator.simulate(
    estimators=ests, num_reps=10, alpha=0.05, show_progress=False
)

Simulators:   0%|          | 0/3 [00:00<?, ?it/s]

Estimators:   0%|          | 0/4 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

...

Estimators:   0%|          | 0/4 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
means = result.means()
display(means)

,name,estimator,prop_signif,sample_rmse,pop_rmse,type_s_rate,rank_corr
0,chick_simulator,Difference,0.463158,0.056689,0.077886,0.000000,0.637444
1,chick_simulator,Exposed Only,0.639474,0.039692,0.066056,0.000000,0.756122
2,chick_simulator,Posterior Mean,0.776316,0.033123,0.045939,0.002632,0.792713
3,chick_simulator,Posterior Mean (Estimated HP),1.000000,0.054158,0.010396,0.044737,0.782974
4,p_star_estimated,Difference,0.186842,0.114433,0.130377,0.005263,0.431404
5,p_star_estimated,Exposed Only,0.773684,0.031524,0.065484,0.002632,0.845432
6,p_star_estimated,Posterior Mean,0.694737,0.031700,0.042337,0.000000,0.882110
7,p_star_estimated,Posterior Mean (Estimated HP),1.000000,0.049757,0.011538,0.023684,0.869395
8,p_star_oracle,Difference,0.215789,0.105966,0.113635,0.007895,0.363169
9,p_star_oracle,Exposed Only,0.713158,0.031237,0.066132,0.000000,0.863027


Notice that the adaptive procedures (`p_star_oracle` and `p_star_estimated`) have smaller RMSEs than the constant $p=0.5$ simulator (`chick_simulator`).

Below we fix $\sigma_0 = \sigma_1$ to the original values for the chicken data and vary $\sigma_b$ from 0 to 0.1.

In [11]:
sims = []
for sigma_b in utils.CHICK_SIGMA_B_GRID:
    for original_sim in original_sims:
        sim = deepcopy(original_sim)
        sim.params["sigma_b"] = sigma_b
        sim.identifier["sigma_b"] = sigma_b
        sims.append(sim)

sigma_b_aggregator = aggregators.SimulationAggregator(simulators=sims)

In [12]:
sigma_b_result = sigma_b_aggregator.simulate(
    estimators=ests,
    num_reps=10,
    alpha=0.05,
    show_progress=False,
)

Simulators:   0%|          | 0/33 [00:00<?, ?it/s]

Estimators:   0%|          | 0/4 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

...

Estimators:   0%|          | 0/4 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

Repetition:   0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
sigma_b_means = sigma_b_result.means()
display(sigma_b_means)

,name,sigma_b,estimator,prop_signif,sample_rmse,pop_rmse,type_s_rate,rank_corr
0,chick_simulator,0.00,Difference,0.394737,0.054694,0.080054,0.002632,0.696313
1,chick_simulator,0.00,Exposed Only,0.628947,0.039852,0.075533,0.000000,0.814181
2,chick_simulator,0.00,Posterior Mean,0.760526,0.032545,0.037445,0.000000,0.783981
3,chick_simulator,0.00,Posterior Mean (Estimated HP),1.000000,0.053706,0.012434,0.052632,0.778904
4,chick_simulator,0.01,Difference,0.450000,0.056191,0.081656,0.010526,0.695218
...,...,...,...,...,...,...,...,...
127,p_star_oracle,0.09,Posterior Mean (Estimated HP),1.000000,0.052382,0.016499,0.063158,0.480425
128,p_star_oracle,0.10,Difference,0.381579,0.067485,0.093038,0.002632,0.674209
129,p_star_oracle,0.10,Exposed Only,0.668421,0.107136,0.123662,0.057895,0.474122
130,p_star_oracle,0.10,Posterior Mean,0.642105,0.043212,0.044946,0.007895,0.690141


In [ ]:
import plotly.express as px

posterior_sigma_b_means = sigma_b_means[sigma_b_means["estimator"] == "Posterior Mean"]
sigma0 = sigma_b_means["R"]

# define dash styles for the two p‐settings
dash_map = {"p = constant 0.5": "solid", "p = 0.5 to 0.95": "dash"}

fig_sample_rmse = px.line(
    posterior_sigma_b_means,
    x="sigma_b",
    y="sample_rmse",
    color="name",
    markers=True,
    title="RMSE vs. sigma_b by Estimator",
)

# keep legend showing only estimator
fig_sample_rmse.update_layout(legend_title_text="Estimator")
fig_sample_rmse.show()

In [21]:
R = np.linspace(0.05, 800, 30)
sigma_0s = []
sims_r = []
for r in R:
    for original_sim in original_sims:
        sim = deepcopy(original_sim)
        sim.identifier["r"] = r
        sim.params["sigma0"] = np.sqrt(64 * r) * utils.CHICK_SIGMA_B
        sim.params["sigma1"] = sim.params["sigma0"]
        sigma_0s.append(sim.params["sigma0"])
        sims_r.append(sim)
r_aggregator = aggregators.SimulationAggregator(simulators=sims_r)

In [26]:
r_result = r_aggregator.simulate(
    estimators=[posterior_mean, exposed_only, difference],
    num_reps=5,
    alpha=0.05,
    show_progress=False,
)

Simulators:   0%|          | 0/90 [00:00<?, ?it/s]

Estimators:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/5 [00:00<?, ?it/s]

Repetition:   0%|          | 0/5 [00:00<?, ?it/s]

Repetition:   0%|          | 0/5 [00:00<?, ?it/s]

...

Repetition:   0%|          | 0/5 [00:00<?, ?it/s]

Estimators:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/5 [00:00<?, ?it/s]

Repetition:   0%|          | 0/5 [00:00<?, ?it/s]

Repetition:   0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
r_means = r_result.means()
display(r_means)

,name,r,estimator,prop_signif,sample_rmse,pop_rmse,type_s_rate,rank_corr
0,chick_simulator,0.050000,Difference,0.442105,0.053051,0.073801,0.000000,0.657468
1,chick_simulator,0.050000,Exposed Only,0.636842,0.041778,0.071789,0.005263,0.787810
2,chick_simulator,0.050000,Posterior Mean,0.726316,0.036006,0.046690,0.000000,0.819280
3,chick_simulator,27.634483,Difference,0.405263,0.060527,0.086904,0.005263,0.709071
4,chick_simulator,27.634483,Exposed Only,0.605263,0.038335,0.066798,0.005263,0.795864
...,...,...,...,...,...,...,...,...
265,p_star_oracle,772.415517,Exposed Only,0.557895,0.044850,0.071625,0.000000,0.721414
266,p_star_oracle,772.415517,Posterior Mean,0.684211,0.043930,0.041367,0.010526,0.712441
267,p_star_oracle,800.000000,Difference,0.115789,0.181678,0.189153,0.021053,0.243944
268,p_star_oracle,800.000000,Exposed Only,0.578947,0.045885,0.068882,0.000000,0.730342


In [ ]:
import plotly.express as px

# Filter for the Posterior Mean estimator
posterior_r_means = r_means[r_means["estimator"] == "Posterior Mean"]
posterior_r_means["sigma0"] = np.sqrt(64 * r_means["r"]) * utils.CHICK_SIGMA_B

# Plot sample RMSE vs r for each simulator
fig_r_sample_rmse = px.line(
    posterior_r_means,
    x="sigma0",
    y="sample_rmse",
    color="name",
    markers=True,
    title="Sample RMSE vs sigma0=sigma1 by Simulator (Posterior Mean)",
)

fig_r_sample_rmse.update_layout(
    xaxis_title="sigma0", yaxis_title="Sample RMSE", legend_title_text="Simulator"
)

fig_r_sample_rmse.show()

/var/folders/fb/h3lz2z7j14j7hndqzb5_3cth0000gn/T/ipykernel_65315/530565250.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
exposed_only_means = r_means[r_means["estimator"] == "Exposed Only"]

fig_exposed_only = px.line(
    exposed_only_means,
    x="sigma0",
    y="sample_rmse",
    color="name",
    markers=True,
    title="Sample RMSE vs sigma0=sigma1 by Simulator (Exposed Only)",
)
fig_exposed_only.update_layout(
    xaxis_title="sigma0", yaxis_title="Sample RMSE", legend_title_text="Simulator"
)

In [ ]:
r_grid_p_star = np.linspace(1, 500, 6)
p_grid_p_star = np.linspace(0.05, 0.95, 10)
sims_r_p_star = []
for r in r_grid_p_star:
    for p in p_grid_p_star:
        sim = deepcopy(utils.CHICK_SIMULATOR)
        sim.
        sim.identifier["r"] = r
        sim.params["sigma0"] = np.sqrt(64 * r) * utils.CHICK_SIGMA_B
        sim.params["sigma1"] = sim.params["sigma0"]
        sims_r_p_star.append(sim)



In [ ]:
def posterior_variance_theta(params):
    sigma1 = params["sigma1"]
    sigma0 = params["sigma0"]
    p = params["p"][0]
    n = params["n"][0]
    sigma_theta = params["sigma_theta"]
    sigma_b = params["sigma_b"]

    V1 = sigma1**2 / (p * n)
    V0 = sigma0**2 / ((1 - p) * n)

    A = 1.0 / V1 + 1.0 / (sigma_theta**2)
    B = 1.0 / V1 + 1.0 / V0 + 1.0 / (sigma_b**2)
    C = 1.0 / V1

    denom = A * B - C**2
    theta_post_var = B / denom

    return theta_post_var

In [ ]:
p_grid = np.linspace(0.05, 0.95, 3)
num_reps = 3
basic_mlm = sim.StanFitter(CmdStanModel(stan_file="../basic_mlm.stan"))
basic_mlm_with_hyperparams = sim.StanFitter(
    CmdStanModel(stan_file="../basic_mlm_hp.stan")
)
params = {
    "n": np.array([sim.CHICK_N]),
    "mu_b": 0,
    "mu_theta": sim.CHICK_MU_THETA,
    "sigma_b": sim.CHICK_SIGMA_B,
    "sigma_theta": sim.CHICK_SIGMA_THETA,
    "sigma1": sim.CHICK_SIGMA1,
    "sigma0": sim.CHICK_SIGMA0,
}

In [ ]:
variances = pd.DataFrame(index=p_grid)

In [ ]:
avg_variances_no_hp = {}

for p in tqdm(p_grid, desc="p", leave=False):
    variances_no_hp = []

    for _ in tqdm(range(num_reps), desc="num_reps", leave=False):
        params["p"] = np.array([p])
        df = sim.simulate_experiments(params)
        print(sim.expt_df_to_dict(df))
        fit = basic_mlm.sample(data=sim.expt_df_to_dict(df), show_progress=False)
        theta_variance = np.var(fit.theta)
        variances_no_hp.append(theta_variance)

    avg_variances_no_hp[p] = np.mean(variances_no_hp)

In [ ]:
variances["unknown hyperparameters"] = [avg_variances_no_hp[p] for p in p_grid]

In [ ]:
avg_variances_hp = {}

for p in tqdm(p_grid, desc="p", leave=False):
    variances_hp = []

    for _ in tqdm(range(num_reps), desc="num_reps", leave=False):
        params["p"] = np.array([p])

        df = sim.simulate_experiments(params)
        dict_df = sim.expt_df_to_dict(df)
        dict_df["mu_b"] = params["mu_b"]
        dict_df["sigma_b"] = params["sigma_b"]
        dict_df["mu_theta"] = params["mu_theta"]
        dict_df["sigma_theta"] = params["sigma_theta"]

        fit = model_hp.sample(data=dict_df, show_progress=False)
        theta_variance = np.var(fit.theta)
        variances_hp.append(theta_variance)

    avg_variances_hp[p] = np.mean(variances_hp)

In [ ]:
variances["known hyperparameters"] = [avg_variances_hp[p] for p in p_grid]

In [ ]:
theoretical_variances_hp = []
for p in p_grid:
    params["p"] = np.array([p])
    theta_post_var = posterior_variance_theta(params)
    theoretical_variances_hp.append(theta_post_var)

In [ ]:
variances["theoretical variance"] = theoretical_variances_hp

In [ ]:
min_index = variances["unknown hyperparameters"].idxmin()
min_value = variances["unknown hyperparameters"].min()

fig = px.line(
    variances.reset_index(),
    x="index",
    y=variances.columns,
    labels={"index": "p", "value": "Variance"},
    title=f"Posterior Variance of Theta vs p (sigma_0 = sigma_1 = {sim.CHICK_SIGMA_THETA:.3f})",
)

# Find the minimum for "known hyperparameters"
min_index_known = variances["known hyperparameters"].idxmin()
min_value_known = variances["known hyperparameters"].min()

# Add a red dot at the minimum point for "unknown hyperparameters"
fig.add_scatter(
    x=[min_index],
    y=[min_value],
    mode="markers",
    marker=dict(color="blue", size=10),
    name="Minimum (unknown hyperparameters)",
)

# Add a blue dot at the minimum point for "known hyperparameters"
fig.add_scatter(
    x=[min_index_known],
    y=[min_value_known],
    mode="markers",
    marker=dict(color="red", size=10),
    name="Minimum (known hyperparameters)",
)

# Add a red dot at the minimum point
fig.update_yaxes(type="log")

fig.update_layout(xaxis_title="p", yaxis_title="Variance", legend_title="")
fig.show()

In [ ]:
def get_sigma_b_hat(expt_df):
    y_1 = expt_df["avg_treated_response"]
    y_0 = expt_df["avg_control_response"]
    y_1_bar = np.mean(y_1)
    y_0_bar = np.mean(y_0)
    num_expts = len(expt_df)

    var_b_hat = np.sum(((y_1 - y_1_bar) * (y_0 - y_0_bar))) / num_expts
    if var_b_hat <= 0:
        return 1e-4

    return (var_b_hat) ** 0.5


def get_p_star(expt_df):
    # here I assume all experiments have the same number of subjects
    assert expt_df["n"].nunique() == 1
    n = expt_df["n"].iloc[0]

    sigma_0 = expt_df.attrs["sigma0"]
    sigma_1 = expt_df.attrs["sigma1"]
    sigma_b_hat = get_sigma_b_hat(expt_df)

    stationary_pt = (1 + sigma_0**2 / (n * sigma_b_hat**2)) / (1 + sigma_0 / sigma_1)
    p_star = min(0.95, stationary_pt)
    return p_star

In [ ]:
import copy


def simulate_adaptive_experiments(params):
    params = copy.deepcopy(params)

    n = params["n"]
    num_expts = len(n)

    # Generate initial experiment
    params["n"] = np.array([params["n"][0]])
    params["p"] = np.array([0.5])
    expt_df = sim.simulate_experiments(params)

    for expt_id in tqdm(range(1, num_expts), desc="Experiment", leave=False):
        num_subjects = n[expt_id]
        ratio = get_p_star(expt_df)

        # Generate another experiment
        params["n"] = np.array([num_subjects])
        params["p"] = np.array([ratio])

        # Append to existing expt_df
        new_expt_df = sim.simulate_experiments(params)
        expt_df = pd.concat([expt_df, new_expt_df], ignore_index=True)

    return expt_df

In [ ]:
def posterior_adaptive_mse(model, params, reps):
    balanced_sample_mses, balanced_pop_mses = [], []
    adaptive_sample_mses, adaptive_pop_mses = [], []

    for rep in tqdm(range(reps), desc="Repetition", leave=False):
        params_balanced = copy.deepcopy(params)
        params_adaptive = copy.deepcopy(params)

        params_balanced["p"] = np.full(len(params_balanced["n"]), 0.5)
        balanced_expt_df = sim.simulate_experiments(params_balanced)

        adaptive_expt_df = simulate_adaptive_experiments(params_adaptive)

        balanced_estimates = sim.evaluate_estimates(
            sim.get_bayes_estimates(balanced_expt_df, model, oracle=True)
        )
        adaptive_estimates = sim.evaluate_estimates(
            sim.get_bayes_estimates(adaptive_expt_df, model, oracle=True)
        )

        balanced_sample_mses.append(balanced_estimates["sample_mse"])
        adaptive_sample_mses.append(adaptive_estimates["sample_mse"])

        balanced_pop_mses.append(balanced_estimates["pop_mse"])
        adaptive_pop_mses.append(adaptive_estimates["pop_mse"])

    return pd.DataFrame(
        {
            "balanced_sample_mse": balanced_sample_mses,
            "adaptive_sample_mse": adaptive_sample_mses,
            "balanced_pop_mse": balanced_pop_mses,
            "adaptive_pop_mse": adaptive_pop_mses,
        }
    )

In [ ]:
chick_params = {
    "n": np.array([sim.CHICK_NUM_SUBJECTS_PER_EXPT] * sim.CHICK_NUM_EXPTS),
    "mu_b": 0,
    "mu_theta": sim.CHICK_MU_THETA,
    "sigma_b": sim.CHICK_SIGMA_CONTROL,
    "sigma_theta": sim.CHICK_SIGMA_THETA,
    "sigma1": 3 * sim.CHICK_SIGMA_TREATMENT,
    "sigma0": sim.CHICK_SIGMA_CONTROL,
}

In [ ]:
res = posterior_adaptive_mse(model_no_hp, chick_params, 100)

In [ ]:
print("Adaptive sample MSE: ", res["adaptive_sample_mse"].mean())
print("Adaptive population MSE: ", res["adaptive_pop_mse"].mean())
print()
print("Balanced sample MSE: ", res["balanced_sample_mse"].mean())
print("Balanced population MSE: ", res["balanced_pop_mse"].mean())

In [ ]:
diff_params = copy.deepcopy(chick_params)
diff_params["p"] = np.array([0.75] * sim.CHICK_NUM_EXPTS)

diff_sample_mse = []
diff_pop_mse = []

for _ in range(100):
    expt_df = sim.simulate_experiments(diff_params)

    evaluation_df = sim.evaluate_estimates(
        sim.get_difference_estimates(expt_df, oracle=True)
    )
    sample_mse = evaluation_df["sample_mse"]
    pop_mse = evaluation_df["pop_mse"]

    diff_sample_mse.append(sample_mse)
    diff_pop_mse.append(pop_mse)

In [ ]:
print("Difference sample MSE: ", np.mean(diff_sample_mse))
print("Difference population MSE: ", np.mean(diff_pop_mse))